##Travelling salesman problem

##Problem 1.

###1)	Mr iyer is a salesman with Delite Manufacturing Company. He wants to visit six cities say, 1, 2, 3, 4, 5 and 6, starting with city 1 where he is stationed. The distance between various cities is given in the Table below. Mr Iyer wants to develop a tour through the five other cities and return to his home city in such a way that he has to travel the minimum distance.

In [ ]:
!pip install pulp

     |████████████████████████████████| 40.6MB 100kB/s 


In [ ]:
from pulp import *
import numpy as np
import pandas as pd

##Importing csv files in google collab

In [ ]:
from google.colab import files
data=files.upload()
for x in data.keys():
  print(f"User uploaded file {x} with size {len(data[x])} bytes")

Saving problem1.csv to problem1.csv
User uploaded file problem1.csv with size 111 bytes


In [ ]:

df=pd.read_csv('problem1.csv')
df

,1,2,3,4,5,6
0,0,10,100,50,33,66
1,10,0,22,86,952,3
2,100,22,0,6,86,2
3,50,86,6,0,5,4
4,33,952,86,5,0,9
5,66,3,2,4,9,0


In [ ]:
nvar=6*6
city=['1','2','3','4','5','6']
dist=df.iloc[0:6,0:6].to_numpy()
cost = dict(((a,b),dist[ord(a)-48-1][ord(b)-48-1]) for a in city for b in city if a!=b)

###define Lpp problem

In [ ]:
model=LpProblem("lp",LpMinimize)

###Define decision variable

In [ ]:
x = LpVariable.dicts('x',cost, 0,1,LpBinary)

##define objective function

In [ ]:
model+=lpSum([x[(i,j)]*cost[(i,j)] for (i,j) in cost])

##adding constraints

In [ ]:
for k in city:
    model+= lpSum([ x[(i,k)] for i in city if (i,k) in x]) ==1 ##i to j
    model+=lpSum([ x[(k,i)] for i in city if (k,i) in x]) ==1  ##j to i

##Linking subtours

In [ ]:
z = LpVariable.dicts('z', city, 0, len(city)-1, LpInteger)
for i in city:
    for j in city:
        if i != j and (i != '1' and j!= '1') and (i,j) in x:
            model += z[i] - z[j] <= (6)*(1-x[(i,j)]) - 1
model

lp:
MINIMIZE
10*x_('1',_'2') + 100*x_('1',_'3') + 50*x_('1',_'4') + 33*x_('1',_'5') + 66*x_('1',_'6') + 10*x_('2',_'1') + 22*x_('2',_'3') + 86*x_('2',_'4') + 952*x_('2',_'5') + 3*x_('2',_'6') + 100*x_('3',_'1') + 22*x_('3',_'2') + 6*x_('3',_'4') + 86*x_('3',_'5') + 2*x_('3',_'6') + 50*x_('4',_'1') + 86*x_('4',_'2') + 6*x_('4',_'3') + 5*x_('4',_'5') + 4*x_('4',_'6') + 33*x_('5',_'1') + 952*x_('5',_'2') + 86*x_('5',_'3') + 5*x_('5',_'4') + 9*x_('5',_'6') + 66*x_('6',_'1') + 3*x_('6',_'2') + 2*x_('6',_'3') + 4*x_('6',_'4') + 9*x_('6',_'5') + 0
SUBJECT TO
_C1: x_('2',_'1') + x_('3',_'1') + x_('4',_'1') + x_('5',_'1') + x_('6',_'1')
 = 1

_C2: x_('1',_'2') + x_('1',_'3') + x_('1',_'4') + x_('1',_'5') + x_('1',_'6')
 = 1

_C3: x_('1',_'2') + x_('3',_'2') + x_('4',_'2') + x_('5',_'2') + x_('6',_'2')
 = 1

_C4: x_('2',_'1') + x_('2',_'3') + x_('2',_'4') + x_('2',_'5') + x_('2',_'6')
 = 1

_C5: x_('1',_'3') + x_('2',_'3') + x_('4',_'3') + x_('5',_'3') + x_('6',_'3')
 = 1

_C6: x_('3',_'1') + x_

##solving...

In [ ]:
status=model.solve()

print(f"status: {model.status},{LpStatus[model.status]}")
print(f"Objective= {model.objective.value()}")

for var in x.values():
  print("{}={}".format(var.name,var.value()))

status: 1,Optimal
Objective= 59.0
x_('1',_'2')=0.0
x_('1',_'3')=0.0
x_('1',_'4')=0.0
x_('1',_'5')=1.0
x_('1',_'6')=0.0
x_('2',_'1')=1.0
x_('2',_'3')=0.0
x_('2',_'4')=0.0
x_('2',_'5')=0.0
x_('2',_'6')=0.0
x_('3',_'1')=0.0
x_('3',_'2')=0.0
x_('3',_'4')=0.0
x_('3',_'5')=0.0
x_('3',_'6')=1.0
x_('4',_'1')=0.0
x_('4',_'2')=0.0
x_('4',_'3')=1.0
x_('4',_'5')=0.0
x_('4',_'6')=0.0
x_('5',_'1')=0.0
x_('5',_'2')=0.0
x_('5',_'3')=0.0
x_('5',_'4')=1.0
x_('5',_'6')=0.0
x_('6',_'1')=0.0
x_('6',_'2')=1.0
x_('6',_'3')=0.0
x_('6',_'4')=0.0
x_('6',_'5')=0.0


##get the results

In [ ]:
starting_city = '1'
s_route=[]##shortest route
s_route.append(city.pop(0))

while len(city) > 0:

    for k in city:
        if x[(starting_city,k)].varValue ==1:
            s_route.append( city.pop( city.index(k)))
            starting_city=k
            break

s_route.append('1')

shortest_route_length = [cost[(s_route[i-1], s_route[i])] for i in range(1,len(s_route))]

print('Shortest Path  Travelling by Mr.Iyer :')
print(' -> '.join(s_route))
print(f"Minimum cost of the tour for mr.Iyer from his home city to  travel other 5 cities with min cost and returning back to its home city :{sum(shortest_route_length)}")

Shortest Path  Travelling by Mr.Iyer :
1 -> 5 -> 4 -> 3 -> 6 -> 2 -> 1
Minimum cost of the tour for mr.Iyer from his home city to  travel other 5 cities with min cost and returning back to its home city :59


#Shortest Path  Travelling by Mr.Iyer :
#1 -> 5 -> 4 -> 3 -> 6 -> 2 -> 1
#Minimum cost of the tour for mr.Iyer from his home city to  travel other 5 cities with min cost and returning back to its home city : $59$

In [ ]:
from pulp import *
import numpy as np

In [ ]:
#a handful of sites
sites = ['org','A','B','C','D','E','F','G','H','I','J','K']

In [ ]:
#non symetric distances
distances = dict( ((a,b),np.random.randint(10,50)) for a in sites for b in sites if a!=b )

In [ ]:
#create the problme
prob=LpProblem("salesman",LpMinimize)

In [ ]:
#indicator variable if site i is connected to site j in the tour
x = LpVariable.dicts('x',distances, 0,1,LpBinary)

In [ ]:
#the objective
cost = lpSum([x[(i,j)]*distances[(i,j)] for (i,j) in distances])
prob+=cost

In [ ]:
#constraints
for k in sites:
    #every site has exactly one inbound connection
    prob+= lpSum([ x[(i,k)] for i in sites if (i,k) in x]) ==1
    #every site has exactly one outbound connection
    prob+=lpSum([ x[(k,i)] for i in sites if (k,i) in x]) ==1

In [ ]:
#we need to keep track of the order in the tour to eliminate the possibility of subtours
u = LpVariable.dicts('u', sites, 0, len(sites)-1, LpInteger)

In [ ]:
#subtour elimination
N=len(sites)
for i in sites:
    for j in sites:
        if i != j and (i != 'org' and j!= 'org') and (i,j) in x:
            prob += u[i] - u[j] <= (N)*(1-x[(i,j)]) - 1

In [ ]:
%time prob.solve()
print(LpStatus[prob.status])

CPU times: user 9.34 ms, sys: 1.96 ms, total: 11.3 ms
Wall time: 77.4 ms
Optimal


In [ ]:
sites_left = sites.copy()
org = 'org'
tour=[]
tour.append(sites_left.pop( sites_left.index(org)))

while len(sites_left) > 0:
    
    for k in sites_left:
        if x[(org,k)].varValue ==1:
            tour.append( sites_left.pop( sites_left.index(k)))
            org=k
            break
            
tour.append('org')

tour_legs = [distances[(tour[i-1], tour[i])] for i in range(1,len(tour))]

print('Found optimal tour!')
print(' -> '.join(tour))

Found optimal tour!
org -> D -> E -> B -> F -> C -> H -> K -> A -> G -> I -> J -> org


In [ ]:
sum(tour_legs)

172